In [2]:
import re
from io import StringIO
import time
import pandas as pd
import requests
import jsonpath_ng as jp



 ## Acceso a Uniprot API

In [ ]:


# Define the base URL for the Uniprot API
base_url = "https://rest.uniprot.org/uniprotkb/search"
# Define the parameters for the API request
params = {
  # TP53 in mammals (taxonomy ID 40674)
  "query": "(reviewed:true) AND (taxonomy_id:40674) AND (gene:TP53)",
  "format": "tsv",
  "fields": "accession,id,sequence,protein_name,organism_name,go",
  "size": 10
}

# Make the API request
response = requests.get(base_url, params=params, timeout=30)

response.raise_for_status()



 ### Extract and print protein information

In [ ]:
data = pd.read_csv(
  StringIO(response.content.decode("utf-8")),
  sep="\t",
  header=0
)

print(data)



 ### Obtener el resto de los resultados

In [ ]:
pattern = re.compile(".*<(.+)>.*$")

match = re.match(pattern, response.headers["link"])

if match:
  next_page_url = match.group(1)
  print(next_page_url)
else:
  print("No match found in the link header.")

In [ ]:


response = requests.get(next_page_url, timeout=30)

response.raise_for_status()
data = pd.read_csv(
  StringIO(response.content.decode("utf-8")),
  sep="\t",
  header=0
)
print(data)



 ### Formatos de salida
 | Formato                              | Tipo MIME                   | Ext.  |
 |--------------------------------------|-----------------------------|------ |
 | JavaScript Object Notation (JSON)    | application/json            | json  |
 | Extensible Markup Language (XML)     | application/xml             | xml   |
 | Text file representation             | text/plain; format=flatfile | txt   |
 | List of one or more IDs              | text/plain; format=list     | list  |
 | Tab-Separated-Values                 | text/plain; format=tsv      | tsv   |
 | FASTA: a text-based                  | text/plain; format=fasta    | fasta |
 | Genomic Feature Format (GFF)         | text/plain; format=gff      | gff   |
 | Open Biomedical Ontologies (OBO)     | text/plain; format=obo      | obo   |
 | Resource Description Framework (RDF) | application/rdf+xml         | rdf   |
 | Excel                                | application/vnd.ms-excel    | xlsx  |



 ### Campos disponibles
 | Campo                   | Descripción                           |
 |-------------------------|---------------------------------------|
 | accession               | Acceso principal de la entrada        |
 | id                      | Nombre del entry (como `INSR_HUMAN`)  |
 | protein_name            | Nombre de la proteína                 |
 | gene_primary            | Nombre principal del gen              |
 | gene_synonym            | Sinónimos del gen                     |
 | organism_name           | Nombre del organismo                  |
 | organism_id             | ID taxonómico (NCBI) del organismo    |
 | length                  | Longitud de la secuencia              |
 | sequence                | Secuencia de aminoácidos completa     |
 | function / cc_function  | Función biológica anotada             |
 | cc_subcellular_location | Localización subcelular               |
 | cc_pathway              | Rutas metabólicas en las que participa|
 | go                      | Anotaciones Gene Ontology (todas)     |
 | xref_pdb                | Referencias a estructuras 3D (PDB)    |
 | xref_geneid             | Enlace al NCBI Gene ID                |
 | xref_kegg               | Enlace a la base de datos KEGG        |
 | ft_binding              | Sitios de unión                       |
 | ft_active               | Sitios activos                        |
 | ft_domain               | Dominios anotados                     |

 Lista completa: [https://www.uniprot.org/help/return_fields](https://rest.uniprot.org/uniprotkb/search/fields)

In [ ]:


base_url = "https://rest.uniprot.org/uniparc/search"
# Define the parameters for the API request
params = {
  # TP53 in mammals (taxonomy ID 40674)
  "query": "(taxonomy_id:40674) AND (gene:TP53)",
  "format": "json",
  "fields": "accession,sequence",
  "size": 10
}

# Make the API request
response = requests.get(base_url, params=params, timeout=30)

response.raise_for_status()

data = response.json()

In [ ]:


sequences = [x.value for x in jp.parse("$..sequence.value").find(data)]
print(sequences)

In [ ]:


base_url = "https://rest.uniprot.org/uniref/search"
# Define the parameters for the API request
params = {
  "query": "(taxonomy_id:9606) AND (name:FUS) AND (identity:0.5)",
  "format": "json",
  "fields": "id,members",
  "size": 10
}

# Make the API request
response = requests.get(base_url, params=params, timeout=30)

response.raise_for_status()

data = response.json()
print(data)

In [ ]:


sequences = [x.value for x in jp.parse("$..members").find(data)]
print(sequences)



 ## Ejercicios
 1. Recuperar de UniprotKB todas las proteínas del género *Caenorhabditis* que
    estén anotadas con el E.C. number 1.1.1.1 (alcohol dehidrogenasa).
    Recuperar los campos `accession`, `id`, `sequence`, `protein_name`,
    `organism_name`, `gene_name` y `reviewed`, en formato TSV.



 2. Recuperar de UniprotKB todas las proteínas que interactúan
    con la proteína *BRCA1* en humanos (revisada). Recuperar además los campos
   `accession`, `id`, y las proteínas que interactúan con cada una de ellas.
    Generar una matriz de interacción, en formato TSV.



 3. Recuperar de swissprot todas las proteínas humanas cuya ubicación
    subcelular se la mitocondria y el peroxisoma (que contengan ambas
    anotaciones), en formato TSV.



 4. Recuperar de UniprotKB todas DNA helicasas de bacterias del grupo
    Cyanobacteriota.
    Recuperar los el accession, el id, la secuencia, el nombre de la proteína,
    el nombre del organismo, el nombre del gen y la longitud de la secuencia.



 5. Recuperar de Uniref los grupos de proteínas al 50% de identidad que
    contengan, alguna de las isoformas humanas de la enzima
    phosphofructokinase. ¿Las isoformas humanas están en el mismo grupo?
    ¿Cuantos organismos diferentes hay en cada grupo?



 6. Recuperar de UniprotKb las anotaciones de Regiones, dominios y Zinc-finger
    de todas las proteínas del gene *FUS* (revisadas).
    Para cada entrada, generar un tabla que contenga estas anotaciones.

7. Recuperar los Uniprot AC de los miembros del clusters UniRef50 en el que se encuentra la proteía "HELI_NPVAC".
   Para cada miembro recuperar el organismo y el linaje taxonomico y si son parte de swissprot.



 ## Uniprot Mapping ID API

In [ ]:


base_url = "https://rest.uniprot.org/idmapping/run"

params = {
  "from": "UniProtKB_AC-ID",
  "to": "PDB",
  "ids": "P12345,Q8N158"
}

response = requests.post(base_url, data=params, timeout=30)
response.raise_for_status()

job_id = response.json()["jobId"]

In [ ]:


# Check the status of the job
status_url = f"https://rest.uniprot.org/idmapping/status/{job_id}"
while True:
  time.sleep(2)
  status_response = requests.get(status_url, timeout=30, allow_redirects=False)
  status_response.raise_for_status()
  status_data = status_response.json()
  if "jobStatus" not in status_data:
    print("Job status not found in response.")
    print(status_data)
    break
  if status_data["jobStatus"] == "FINISHED":
    break

In [ ]:


# Retrieve the results
results_url = f"https://rest.uniprot.org/idmapping/results/{job_id}"
results_response = requests.get(results_url, timeout=30)
results_response.raise_for_status()

# Parse and display the results
results_data = results_response.json()
for result in results_data["results"]:
  print(f"From: {result['from']} -> To: {result['to']}")



 ## Biomart Service

In [ ]:
url = 'http://www.ensembl.org/biomart/martservice'

query = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE Query>
<Query virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >
	<Dataset name = "hsapiens_gene_ensembl" interface = "default" >
		<Filter name = "entrezgene_id" value = "207"/>
		<Attribute name = "ensembl_gene_id" />
		<Attribute name = "ensembl_gene_id_version" />
		<Attribute name = "ensembl_transcript_id" />
		<Attribute name = "ensembl_transcript_id_version" />
	</Dataset>
</Query>"""

query = query.replace("\n", "").replace("\t", "")

params = {
  "query": query
}

response = requests.get(url, params, timeout=30)

print(response.request.url)

response.raise_for_status()

data = response.content.decode("utf-8")

In [ ]:
pd.read_csv(
  StringIO(data),
  sep="\t",
  header=None
)



 ## Ejercicios Uniprot Mapping y Biomart



 1. Recuperar todas las todos los identificadores de UniProtKB de la proteína
    codificada en el gen *ATK1* en humanos, Mus musculus, Drosophila
    melanogaster, Danio rerio y Sus scrofa. Primero recuperar los Gene IDs de
    NCBI Gene y luego mapearlos a UniProtKB, usando la API de Uniprot Mapping.



 2. Repetir el ejercicio anterior, pero usando la API de Biomart. Comparar los
    resultados obtenidos con ambos métodos.



 3. Recuperar las secuencias de todos los exones de todos los transcriptos del
    gen *TP53* humano.

4. Recuperar los nombres de todos los genes humanos que están asociados con el
   término GO "apoptosis" (GO:0006915), junto con sus identificadores NCBI Gene ID
   y el Gene symbol, utilizando la API de Biomart.

   ```
